In [1]:
import pandas as pd
from shapely import wkt,geometry
from shapely.geometry import Point
from shapely.geometry import LineString, Polygon
import networkx as nx
import numpy as np

In [2]:
import sys
import os
sys.path.append(os.path.join(os.environ["SUMO_HOME"], 'tools'))
import sumolib

net = sumolib.net.readNet('../map/osm.net.xml', withInternal=True)

In [3]:
def getShape(edge):
    _shape = list()        
    for point in edge.getShape():  
        _shape.append(net.convertXY2LonLat(point[0], point[1]))
    return LineString(_shape)

def getRawShape(edge):
    _shape = list()        
    for point in edge.getRawShape():
        _shape.append(net.convertXY2LonLat(point[0], point[1]))
    return LineString(_shape)

def net2edgedf(net):
    edgelist = []
    for edge in net.getEdges():
        box = edge.getBoundingBox()
        xx = (box[0]+box[2])/2
        yy = (box[1]+box[3])/2
        _pt = net.convertXY2LonLat(xx,yy)
        _id = edge.getID()
        if(edge.isSpecial()):
            _from = ""
            _to = ""
        else:
            _from = edge.getFromNode().getID()
            _to = edge.getToNode().getID()

        _type = edge.getType()
        _
        _function = edge.getFunction()
        if(_function!="internal"):
            _function ="real"

        _laneNumber = edge.getLaneNumber()

        _length = edge.getLength()
        _speed = edge.getSpeed()
        _name = edge.getName()
        _shape = getShape(edge)
        _rawshape =getRawShape(edge)
        _bicycle = edge.allows("bicycle")
        _vehicle = edge.allows("passenger")
        _pedestrian = edge.allows("pedestrian")
        _bus = edge.allows("bus")
        edgelist.append({"id":_id, "from":_from, "to":_to, "laneNumber":_laneNumber,
                          "pedestrian_allow":_pedestrian,"vehicle_allow":_vehicle, "bicyle_allow":_bicycle,
                         "bus_allow":_bus,"speed":_speed,"function":_function, "shape":_shape,"rawshape":_rawshape,
                         "length":_length, "name":_name, "type":_type})
        
    edge_df = pd.DataFrame.from_dict(edgelist)
    edge_df["weight"] = edge_df.apply(lambda row: row.length/row.speed, axis=1)
    return edge_df


In [4]:
edge_df = net2edgedf(net)
edge_df.head()

,id,from,to,laneNumber,pedestrian_allow,vehicle_allow,bicyle_allow,bus_allow,speed,function,shape,rawshape,length,name,type,weight
0,:10981052_0,,,2,False,True,False,True,13.89,internal,LINESTRING (26.683129875645715 58.357070430325...,LINESTRING (26.683067051228388 58.357168533659...,13.65,,,0.982721
1,:10981052_2,,,2,False,True,False,True,13.89,internal,LINESTRING (26.683010180527045 58.357069415535...,LINESTRING (26.683067051228388 58.357168533659...,13.63,,,0.981281
2,:10981055_0,,,2,False,True,False,True,13.89,internal,LINESTRING (26.68373404994909 58.3565337010081...,LINESTRING (26.683796961110687 58.356454234399...,9.54,,,0.686825
3,:10981055_2,,,1,False,True,False,True,3.65,internal,LINESTRING (26.683713801041538 58.356524085576...,LINESTRING (26.683796961110687 58.356454234399...,2.34,,,0.641096
4,:10981055_5,,,1,False,True,False,True,3.65,internal,LINESTRING (26.683675173079823 58.356525067948...,LINESTRING (26.683796961110687 58.356454234399...,2.34,,,0.641096


In [5]:
len(edge_df)

29435

In [6]:
edges = edge_df.copy()
edges = edges[edges["vehicle_allow"]==False]
len(edges)

3

In [7]:
edges

,id,from,to,laneNumber,pedestrian_allow,vehicle_allow,bicyle_allow,bus_allow,speed,function,shape,rawshape,length,name,type,weight
3413,:2977428680_0,,,1,True,False,True,True,7.13,internal,LINESTRING (26.714531526033298 58.341702655810...,LINESTRING (26.71441996238552 58.3417269267965...,8.23,,,1.154278
3414,:2977428680_1,,,1,True,False,True,True,6.30,internal,LINESTRING (26.714531526033298 58.341702655810...,LINESTRING (26.71441996238552 58.3417269267965...,11.09,,,1.760317
27889,321034538#0,2547229670,2977428680,1,True,False,True,True,5.56,real,LINESTRING (26.71639276544829 58.3417116429666...,LINESTRING (26.71639276544829 58.3417116429666...,144.83,,highway.service,26.048561


In [8]:
nodes = set(edges["from"]).union(set(edges["to"]))
nodes

{'', '2547229670', '2977428680'}

In [9]:
nodes.remove("")

In [10]:
nodes

{'2547229670', '2977428680'}

In [11]:
def getShape(edge):
    _shape = list()        
    for point in edge.getShape():  
        _shape.append(net.convertXY2LonLat(point[0], point[1]))
    return LineString(_shape)

In [12]:
EPSILON = 0.0000001

def getInteralEdgeid(fromedgeid , toedgeid, net):
    fromedge = net.getEdge(fromedgeid)
    toedge = net.getEdge(toedgeid)
    conlist = fromedge.getOutgoing()[toedge]
    return net.getLane(conlist[0].getViaLaneID()).getEdge().getID()

def outgoingInternalEdge(fromedgeid, net):
    outgoinglist = []
    fromedge = net.getEdge(fromedgeid)
    conlist = fromedge.getOutgoing()
    for edge in conlist:
        toedgeid = edge.getID()
        internaledge = getInteralEdgeid(fromedgeid , toedgeid, net)
        outgoinglist.append({"fromedge":fromedgeid, "toedge":toedgeid, "internaledge":internaledge})
    return outgoinglist


def net2graph(net):
    edges = net2edgedf(net)
    edges = edges[edges["function"]=="real"]

    #edges = edges[edges["vehicle_allow"]==True]
    nodes = set(edges["from"]).union(set(edges["to"]))
    if "" in nodes:
        nodes.remove("")
        
    realedge=edges[["id"]].copy()
    realedge["from"] = realedge["id"].apply(lambda x: "from_"+x)
    realedge["to"] = realedge["id"].apply(lambda x: "to_"+x)
    realedge["type"] = "real"
    realedge["speed"] = realedge["id"].apply(lambda x: net.getEdge(x).getSpeed())
    realedge["length"] = realedge["id"].apply(lambda x: net.getEdge(x).getLength())
    
    internaledgelist = []
    templist = []
    for fromedgeid in list(edges["id"]):
        templist = outgoingInternalEdge(fromedgeid, net)
        for item in templist:
            internaledgelist.append(item)   
    internaledge_df = pd.DataFrame(internaledgelist)

    internaledge = internaledge_df.copy()
    internaledge.rename(columns={"fromedge":"from", "toedge":"to", "internaledge":"id"}, inplace=True)
    internaledge["from"] = internaledge["from"].apply(lambda x: "to_"+x)
    internaledge["to"] = internaledge["to"].apply(lambda x: "from_"+x)
    internaledge["type"] = "internal"
    internaledge["speed"] = internaledge["id"].apply(lambda x: net.getEdge(x).getSpeed())
    internaledge["length"] = internaledge["id"].apply(lambda x: net.getEdge(x).getLength())
    
    edge2nodelist = []
    for nodeid in nodes:
        node = net.getNode(nodeid)
        for edge in node.getIncoming():
            if not edge.isSpecial():
                edge2nodelist.append({"fromedge":edge.getID(), "tonode":nodeid})
    edge2node_df = pd.DataFrame(edge2nodelist)

    incomingnode = edge2node_df.copy()
    incomingnode.rename(columns={"fromedge":"from", "tonode":"to"}, inplace=True)
    incomingnode["from"] = incomingnode["from"].apply(lambda x: "to_"+x)
    incomingnode["to"] = incomingnode["to"].apply(lambda x: "d_"+x)
    incomingnode["id"] = incomingnode.apply(lambda row: row["from"] +"_"+row["to"], axis=1)
    incomingnode["type"] = "destination_node"
    incomingnode["speed"] = 1
    incomingnode["length"] = EPSILON
    
    
    node2edgelist = []
    for nodeid in nodes:
        node = net.getNode(nodeid)
        for edge in node.getOutgoing():
            if not edge.isSpecial():
                node2edgelist.append({ "fromnode":nodeid, "toedge":edge.getID()})
    node2edge_df = pd.DataFrame(node2edgelist)

                              
    outgoingnode = node2edge_df.copy()
    outgoingnode.rename(columns={"fromnode":"from", "toedge":"to"}, inplace=True)
    outgoingnode["from"] = outgoingnode["from"].apply(lambda x: "o_"+x)
    outgoingnode["to"]   = outgoingnode["to"].apply(lambda x: "from_"+x)
    outgoingnode["id"]   = outgoingnode.apply(lambda row: row["from"] +"_"+row["to"], axis=1)

    outgoingnode["type"] = "origin_node"
    outgoingnode["speed"] = 1
    outgoingnode["length"] = EPSILON
    
    new_edge_data = pd.concat([realedge, internaledge,incomingnode,outgoingnode])
    new_edge_data["cost"] = new_edge_data["length"]/new_edge_data["speed"]
    
    G = nx.from_pandas_edgelist(new_edge_data, source="from", target="to", edge_attr=True, create_using=nx.DiGraph)
    return G

In [13]:
G = net2graph(net)

In [14]:
G.number_of_edges()

38724

In [15]:
yy = nx.to_edgelist(G)

In [16]:
pd.to_pickle(yy, "graph_edgelist.pkl")

In [17]:
#zz = pd.read_pickle("graph.pkl")

In [18]:
hhh= nx.from_edgelist(yy, create_using=nx.DiGraph)

In [19]:
hhh.is_directed()

True

In [20]:


def DAG_all2destination(G, destination_id):
    G_reverse = nx.reverse(G)
    ss1 = nx.single_source_dijkstra_path_length(G_reverse,source="d_"+str(destination_id), weight="cost")
    empty =np.inf
    nx.set_node_attributes(G_reverse, empty, "dist_from_destination")
    nx.set_node_attributes(G_reverse, ss1, "dist_from_destination")
  
    def filter_edge(n1, n2):
        if G_reverse.nodes[n1]["dist_from_destination"] ==np.inf:
            return False
        elif G_reverse.nodes[n2]["dist_from_destination"] ==np.inf:
            return False
        else:
            return G_reverse.nodes[n1]["dist_from_destination"] <= G_reverse.nodes[n2]["dist_from_destination"]

    def filter_node(n):
        return  G_reverse.nodes[n]["dist_from_destination"] !=np.inf

    view = nx.subgraph_view(G_reverse, filter_edge=filter_edge, filter_node=filter_node)
    DAG_reverse = view.copy()
    DAG = nx.reverse(DAG_reverse)
    return DAG

def filter_DAG_origin_based(DAG, origins):
    nodesorted = sorted(DAG.nodes, key=lambda n: DAG.nodes[n]['dist_from_destination'])
    nodesorted.reverse()
    poi = {"o_"+item for item in origins}
    nodedict = dict()
    for node in nodesorted:
        if (node in nodedict.keys()) or (node in poi):
            nodedict[node]=True
            neighbors_temp = list(nx.neighbors(DAG,node))
            for neighbor in neighbors_temp:
                nodedict[neighbor]=True
        else:
            nodedict[node]=False
            
    def filter_node_DAG(n):
        return  nodedict[n]

    view_DAG = nx.subgraph_view(DAG, filter_node=filter_node_DAG)
    DAG_filtered = view_DAG.copy()
    return DAG_filtered
 

In [21]:
def DAG_all2destination_od(G, destination_id, od):
    origins = set(od["from_node"])
    if destination_id in origins:
        origins.remove(destination_id)
    DAG = DAG_all2destination(G, destination_id)
    DAG_filtered = filter_DAG_origin_based(DAG, origins)
    sorted_nodes = sorted(DAG_filtered.nodes, key=lambda n: DAG_filtered.nodes[n]['dist_from_destination'])
    return DAG_filtered, sorted_nodes

In [22]:
od = pd.read_csv("../od/od.csv")
destination_id = od["to_node"][0]
DAG_filtered, sorted_nodes = DAG_all2destination_od(G, destination_id, od)

In [364]:
DAG_filtered.number_of_edges()

2599

In [365]:
sorted_nodes

['d_11231981',
 'to_205001477',
 'from_205001477',
 'to_261675864',
 'from_261675864',
 'to_437964869',
 'to_247838479',
 'from_437964869',
 'to_853813514',
 'from_853813514',
 'to_853813515',
 'from_247838479',
 'to_655075983',
 'to_655075996',
 'from_853813515',
 'to_32405512',
 'to_241113530',
 'from_32405512',
 'from_241113530',
 'to_205001480#2',
 'to_29331563',
 'from_205001480#2',
 'to_205001480#1',
 'from_655075996',
 'to_261675857#0',
 'from_205001480#1',
 'to_853813518',
 'to_205001480#8',
 'from_205001480#8',
 'to_731752841',
 'from_731752841',
 'from_261675857#0',
 'to_261675854#5',
 'to_-731752840#1',
 'from_29331563',
 'to_29430535#0',
 'from_853813518',
 'to_247827743#1',
 'to_321034535',
 'from_261675854#5',
 'to_261675854#2',
 'to_225022465#1',
 'from_247827743#1',
 'o_2546875545',
 'to_247827743#0',
 'from_247827743#0',
 'to_247827747',
 'from_655075983',
 'to_247827734',
 'from_321034535',
 'from_247827734',
 'to_750789303',
 'to_655075985',
 'from_247827747',
 'from

In [338]:
DAG_filtered.nodes['to_422424372']["dist_from_destination"]

320.6411077725666

In [329]:
sorted_nodes.index('to_853813515')

10

In [216]:
nx.to_pandas_edgelist(DAG_filtered)

,source,target,cost,type,length,speed,id
0,from_-1001559572#2,to_-1001559572#2,7.194245e-02,real,2.000000e-01,2.78,-1001559572#2
1,to_-1001559572#2,from_1001559572#2,1.359712e+00,internal,3.780000e+00,2.78,:cluster_9244476150_9244476151_15
2,from_-1033620732#1,to_-1033620732#1,1.587329e+01,real,2.204800e+02,13.89,-1033620732#1
3,to_-1033620732#1,from_-514218579#1,4.240838e-01,internal,4.860000e+00,11.46,:330042796_5
4,from_-1033622461,to_-1033622461,5.574074e+00,real,1.083600e+02,19.44,-1033622461
...,...,...,...,...,...,...,...
2594,o_cluster_330045282_330045762,from_223572256#0,1.000000e-07,origin_node,1.000000e-07,1.00,o_cluster_330045282_330045762_from_223572256#0
2595,o_cluster_330045131_330045136,from_223572254#0,1.000000e-07,origin_node,1.000000e-07,1.00,o_cluster_330045131_330045136_from_223572254#0
2596,o_330040967,from_29981309#0,1.000000e-07,origin_node,1.000000e-07,1.00,o_330040967_from_29981309#0
2597,o_360477894,from_32119073#15,1.000000e-07,origin_node,1.000000e-07,1.00,o_360477894_from_32119073#15


In [24]:
def create_vdict(DAG, sorted_nodes):
    neighbors = nx.to_dict_of_lists(DAG)
    neighbors_cost = dict()
    for node in neighbors.keys():
        tempdict = dict()
        for item in neighbors[node]:
            tempdict[item] = {"edgecost":DAG[node][item]["cost"]}
        neighbors_cost[node] = {"order":sorted_nodes.index(node),
                                "shortestpathcost":DAG.nodes[node]["dist_from_destination"],
                                "neighbors":tempdict}
    return neighbors_cost

In [25]:
expdict[vertex]

NameError: name 'expdict' is not defined

In [26]:
list(nx.neighbors(DAG_filtered, 'to_-223099572#3'))

['from_-618320310', 'from_29981093#0']

In [27]:
DAG_filtered.edges['from_-1001559572#2','to_-1001559572#2']["cost"]

0.07194244604316548

In [28]:
np.exp(1)

2.718281828459045

In [29]:
def find_first_meet(vdict, nodesset):
    if len(nodesset)==0:
        print("Error: there is no node in the nodes set")
        return 0
    while len(nodesset)!=1:
        maxorder = -1
        maxnode = ""
        for node in nodesset:
            temp_order = vdict[node]["order"]
            if temp_order > maxorder:
                maxorder = temp_order
                maxnode = node
        nodesset.remove(maxnode)
        if "vout" not in vdict[maxnode].keys():
            print(maxnode)
            print(vdict[maxnode])
        else:
            nodesset.add(vdict[maxnode]["vout"])
    return nodesset.pop()

In [30]:
def vcost_node2meet(vdict,node,meet):
    cost = 0
    nextnode = node
    while nextnode!=meet:
        nextnode = vdict[nextnode]["vout"]
        cost += vdict[nextnode]["vcost"]
    return cost


In [31]:
def calculate_vcost(vdict, node, teta):
    nodesset = set(vdict[node]["neighbors"].keys())
    if len(nodesset)==1:
        vdict[node]["vcost"] = EPSILON
        mynode = nodesset.pop()
        vdict[node]["vout"] = mynode
        vdict[node]["neighbors"][mynode]["prob"] = 1
    else:  
        meetnode = find_first_meet(vdict, nodesset)
        vdict[node]["vout"] = meetnode
        pi_od = vdict[node]["shortestpathcost"]-vdict[meetnode]["shortestpathcost"]
        expdict = dict()
        sumexp = 0
        for vertex in vdict[node]["neighbors"].keys():
            edgecost = vdict[node]["neighbors"][vertex]["edgecost"]
            cost = edgecost + vcost_node2meet(vdict,vertex,meetnode)# + vdict[meetnode]["shortestpathcost"]
            expdict[vertex] = np.exp(-teta*cost/pi_od)
            sumexp += expdict[vertex]

        for vertex in vdict[node]["neighbors"].keys():
            vdict[node]["neighbors"][vertex]["prob"] = expdict[vertex]/sumexp

        vdict[node]["vcost"] = np.abs(-(pi_od*sumexp)/teta)
    return vdict
    

In [32]:
def calculate_prob_choice(DAG, sorted_nodes, teta):
    vdict = create_vdict(DAG, sorted_nodes)
    #sorted_nodes.remove(sorted_nodes[0])
    destination = sorted_nodes[0]
    for node in sorted_nodes:
        if node!=destination:
            vdict = calculate_vcost(vdict, node, teta)
    return vdict    

In [33]:
test = calculate_prob_choice(DAG_filtered, sorted_nodes, teta=2.5)

In [34]:
test['from_-30067640#3']

{'order': 850,
 'shortestpathcost': 355.26243003065866,
 'neighbors': {'to_-30067640#3': {'edgecost': 5.627069834413247, 'prob': 1}},
 'vcost': 1e-07,
 'vout': 'to_-30067640#3'}

In [35]:
test['from_30089513#0']

{'order': 847,
 'shortestpathcost': 354.6462387065562,
 'neighbors': {'to_30089513#0': {'edgecost': 9.751619870410366, 'prob': 1}},
 'vcost': 1e-07,
 'vout': 'to_30089513#0'}